In [198]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
from tqdm import tqdm

warnings.simplefilter(action="ignore")


def filter_ones(data):
    return data.WEFAC == 1


def filter_more_than_zero(data):
    return data.WEFAC > 0


def filter_all(data):
    return data.WEFAC >= 0


def filter_zeros(data):
    return data.WEFAC == 0


def draw_scaled(data, well):
    columns = ["WWPR", "WOPR", "WGPR", "WBHP", "WTHP", "WGIR", "WWIR"]

    filters = [filter_ones, filter_more_than_zero, filter_all, filter_zeros]
    filter_names = ["WEFAC==1", "WEFAC>0", "WEFAC>=0", "WEFAC==0"]
    sns.set(rc={"figure.figsize": (12, 30)})
    sns.set_theme(style="whitegrid")
    fig, axs = plt.subplots(nrows=4, ncols=1)
    for i, (filt, filt_name) in enumerate(zip(filters, filter_names)):
        cur_data = data[(data.WELL == well) & filt(data)].fillna(0)
        for c in columns:
            cur_data[c] = cur_data[c] / cur_data[columns].max()[c]
        cur_data = cur_data.fillna(0).reset_index()[columns + ["DATE"]]

        for c in columns:
            ax = sns.scatterplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[i])
            sns.lineplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[i])
        plt.legend()
        ax.set(xlabel=f"DATE", ylabel=f"Scaled params")
        ax.title.set_text(filt_name)
    plt.savefig(f"images/merged_scaled_{well}.jpg")
    plt.clf()



def draw_sub(data, well):
    columns = ["WWPR", "WOPR", "WGPR", "WBHP", "WTHP", "WGIR", "WWIR"]

    sns.set(rc={"figure.figsize": (30, 15)})
    sns.set_theme(style="whitegrid")
    fig, axs = plt.subplots(nrows=2, ncols=3)
    cur_data = data[(data.WELL == well) & filter_all(data)].fillna(0).reset_index()[columns + ["DATE"]]
    for i, c in enumerate(columns[:5]):
        row = i//3
        col = i%3
        ax = sns.scatterplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[row][col])
        sns.lineplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[row][col])
        plt.legend()
        ax.set(xlabel=f"DATE", ylabel=c)
        ax.title.set_text(c)
    for c in columns[5:]:
        ax = sns.scatterplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[1][2])
        sns.lineplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[1][2])
    plt.legend()
    ax.set(xlabel=f"DATE", ylabel=" & ".join(columns[5:]))
    ax.title.set_text(" & ".join(columns[5:]))
    plt.savefig(f"images/six_plots_{well}.jpg")
    plt.clf()

In [199]:
for well in tqdm(np.unique(data.WELL)):
    draw_sub(data, well)

  0%|                                                                            | 0/134 [00:00<?, ?it/s]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


['WWPR', 'WOPR', 'WGPR', 'WBHP', 'WTHP', 'WGIR', 'WWIR']


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  1%|▌                                                                   | 1/134 [00:01<02:34,  1.17s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


['WWPR', 'WOPR', 'WGPR', 'WBHP', 'WTHP', 'WGIR', 'WWIR']


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  1%|█                                                                   | 2/134 [00:02<02:50,  1.29s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


...

<Figure size 2160x1080 with 0 Axes>

<Figure size 2160x1080 with 0 Axes>

<Figure size 2160x1080 with 0 Axes>

<Figure size 2160x1080 with 0 Axes>

<Figure size 2160x1080 with 0 Axes>

In [167]:
data = pd.read_excel("./data/Исходник_для_аномалий.xlsx", decimal='.')
columns = ["WWPR", "WOPR", "WGPR", "WBHP", "WTHP", "WGIR", "WWIR"]
for c in columns+['WEFAC']:
    if data[c].dtype=='O':
        data[c] = pd.Series(data[c].astype(str).apply(lambda x: x.replace(',', '.')).astype('float64'))
data

,WELL,DATE,WWPR,WOPR,WGPR,WBHP,WTHP,WGIR,WWIR,WEFAC
0,1,2017-09-12,0.0,0.0,0.0,0.0,139.16,0.0,0,1.0
1,1,2017-09-13,0.0,0.0,0.0,0.0,138.18,0.0,0,1.0
2,1,2017-09-14,0.0,0.0,0.0,0.0,138.18,0.0,0,1.0
3,1,2017-09-15,0.0,0.0,0.0,0.0,136.22,0.0,0,1.0
4,1,2017-09-16,0.0,0.0,0.0,0.0,136.22,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
144338,134,2021-12-27,0.0,0.0,550560.0,0.0,0.00,0.0,0,1.0
144339,134,2021-12-28,0.0,0.0,166773.0,0.0,0.00,0.0,0,1.0
144340,134,2021-12-29,0.0,0.0,462300.0,0.0,0.00,0.0,0,1.0
144341,134,2021-12-30,0.0,0.0,633745.0,0.0,118.00,0.0,0,1.0


In [ ]:
for well in tqdm(np.unique(data.WELL)):
    draw_scaled(data, well)